# 8. 並行與大規模科學計算

In [15]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [16]:
# !ipcluster start -n 4

In [17]:
# !ipcluster nbextension enable

## 8.1 用 IPython 做並行計算`

## 8.2 IPython 並行計算架構
組成：
- Engine
- Controller
 - Hub
 - Scheduler
- View
- Client

## 8.3 並行計算示例

In [19]:
from IPython import parallel

def pow(a, b):
    return a ** b

clients = parallel.Client()
# clients = parallel.Client(profile='testprofile')
print(clients.ids)

[0, 1, 2, 3]


In [20]:
clients.block = True
clients[0].apply(pow, 2, 4)

16

In [21]:
clients[:].apply(pow, 2, 4)

[16, 16, 16, 16]

In [22]:
map(pow, [2, 3, 4, 5], [2, 3, 4, 5])

In [23]:
view = clients.load_balanced_view()
view.map(pow, [2, 3, 4, 5], [2, 3, 4, 5])

[4, 27, 256, 3125]

### 8.3.1 並行裝飾器

In [24]:
from IPython import parallel
clients = parallel.Client()
lbview = clients.load_balanced_view()
lbview.block = True

serial_computation = map(lambda i:i**5, range(26))
list(serial_computation)

[0,
 1,
 32,
 243,
 1024,
 3125,
 7776,
 16807,
 32768,
 59049,
 100000,
 161051,
 248832,
 371293,
 537824,
 759375,
 1048576,
 1419857,
 1889568,
 2476099,
 3200000,
 4084101,
 5153632,
 6436343,
 7962624,
 9765625]

In [25]:
parallel_computation = lbview.map(lambda i: i**5, range(26))
parallel_computation

[0,
 1,
 32,
 243,
 1024,
 3125,
 7776,
 16807,
 32768,
 59049,
 100000,
 161051,
 248832,
 371293,
 537824,
 759375,
 1048576,
 1419857,
 1889568,
 2476099,
 3200000,
 4084101,
 5153632,
 6436343,
 7962624,
 9765625]

In [26]:
@lbview.parallel()
def func_turned_as_parallel(x):
     return x**5
    
func_turned_as_parallel.map(range(26))

[0,
 1,
 32,
 243,
 1024,
 3125,
 7776,
 16807,
 32768,
 59049,
 100000,
 161051,
 248832,
 371293,
 537824,
 759375,
 1048576,
 1419857,
 1889568,
 2476099,
 3200000,
 4084101,
 5153632,
 6436343,
 7962624,
 9765625]

### IPython 的魔法函數

In [27]:
%px print('Hello')

[stdout:0] Hello
[stdout:1] Hello
[stdout:2] Hello
[stdout:3] Hello


In [28]:
%pxconfig --targets 2

In [29]:
%px print('Hello')

Hello


In [30]:
%pxconfig --targets all

In [31]:
%px print('Hello')

[stdout:0] Hello
[stdout:1] Hello
[stdout:2] Hello
[stdout:3] Hello


In [32]:
%px print('Hello')
print('World')

[stdout:0] Hello
[stdout:1] Hello
[stdout:2] Hello
[stdout:3] Hello
World


In [33]:
%px eval('2+3')

Out[0:4]: 5

Out[1:4]: 5

Out[2:5]: 5

Out[3:4]: 5

In [34]:
from IPython import parallel

clients = parallel.Client()
drctview = clients[:]
drctview.activate()

with drctview.sync_imports():
    import numpy
    
drctview.block=True
%px dummymatrix = numpy.random.rand(4,4)
%px eigenvalue = numpy.linalg.eigvals(dummymatrix)
drctview['eigenvalue']

importing numpy on engine(s)


[array([ 1.47429528+0.j        ,  0.32627704+0.13408733j,
         0.32627704-0.13408733j, -0.19999163+0.j        ]),
 array([ 1.58611546, -0.52257306,  0.5996159 ,  0.40439583]),
 array([ 2.08988850+0.j        , -0.27312062+0.j        ,
         0.44491381+0.50592487j,  0.44491381-0.50592487j]),
 array([ 1.57163926+0.j        , -0.42174371+0.j        ,
         0.01855611+0.06880788j,  0.01855611-0.06880788j])]

In [35]:
%pxconfig --noblock 
%autopx

%autopx enabled


In [ ]:
maximum_egnvals = []
for idx in range(50):
    arr = numpy.random.rand(10,10)
    egnvals = numpy.linalg.eigvals(arr)
    maximum_egnvals.append(egnvals[0].real)

In [ ]:
%autopx
%pxconfig --block 

%autopx disabled


In [38]:
%px answer= "The average maximum eigenvalue is: {}".format(sum(maximum_egnvals)/len(maximum_egnvals))
drctview['answer']

['The average maximum eigenvalue is: 4.997459738809983',
 'The average maximum eigenvalue is: 4.9455133913863545',
 'The average maximum eigenvalue is: 4.987094827377075',
 'The average maximum eigenvalue is: 4.942010312257684']

In [39]:
%%px --block --group-outputs=engine
import numpy as np
arr = np.random.random ((4,4))
egnvals = numpy.linalg.eigvals(arr)
print(egnvals)
egnvals.max()
egnvals.min()

[stdout:0] 
[ 1.68819641+0.j          0.20987832+0.j         -0.23693377+0.07667259j
 -0.23693377-0.07667259j]


[output:0]

Out[0:9]: (-0.23693376649965886-0.076672588545708631j)

[stdout:1] 
[ 1.90546311+0.j         -0.79399500+0.j         -0.00304495+0.07102288j
 -0.00304495-0.07102288j]


[output:1]

...

[output:2]

Out[2:10]: (-0.21053228566586529-0.42193259783065884j)

[stdout:3] 
[ 1.73172300+0.j         -0.10385760+0.j          0.12903425+0.37278305j
  0.12903425-0.37278305j]


[output:3]

Out[3:9]: (-0.10385760319828338+0j)

In [40]:
odd_view = clients[1::2]
odd_view.activate("_odd")
%px print("Test Message")

<AsyncResult: execute>

In [41]:
odd_view.block = True
%px print("Test Message")

<AsyncResult: execute>

In [42]:
clients.activate()
%px print("Test Message")
%px_odd print("Test Message")

[stdout:0] Test Message
[stdout:1] Test Message
[stdout:2] Test Message
[stdout:3] Test Message
[stdout:1] Test Message
[stdout:3] Test Message


## 8.4 IPython 高級特性

### 8.4.1 容錯
```
lbvw.retries = 4
或
with lbvw.temp_flags(retries = 4):
    lbvw.apply(task_tobe_retried)
```   
   

### 8.4.2 Load Balance
```
config.taskSchedulerscheme_name = 'leastload'
或
ipcontroller --scheme=lru
```

### 8.4.3 傳遞 物件
```
push / pull
AsyncResult.get()
scatter / gather <= 這就像是 map / reduce
```

In [43]:
import numpy as np
from IPython import parallel
clients = parallel.Client()
drctview = clients[:]
drctview.push(dict(a=1.03234,b=3453))

<AsyncResult: _push>

In [44]:
ar = drctview.pull('a')
ar

<AsyncResult: _pull>

In [45]:
ar.get()

[1.03234, 1.03234, 1.03234, 1.03234]

In [46]:
ar = drctview.pull('b', targets=0)
ar

<AsyncResult: _pull>

In [47]:
ar.get()

3453

In [48]:
ar = drctview.pull(('a','b'))
ar

<AsyncResult: _pull>

In [49]:
ar.get()

[[1.03234, 3453], [1.03234, 3453], [1.03234, 3453], [1.03234, 3453]]

In [50]:
ar = drctview.push(dict(c='speed'))

In [51]:
ar.get()

[None, None, None, None]

In [52]:
drctview.scatter('a',range(16))

<AsyncResult: scatter>

In [53]:
drctview['a']

[range(0, 4), range(4, 8), range(8, 12), range(12, 16)]

In [54]:
ar = drctview.gather('a')

In [55]:
ar.get()

[range(0, 4), range(4, 8), range(8, 12), range(12, 16)]

In [56]:
def paralleldot(vw, mat1, mat2):
    vw['mat2'] = mat2
    vw.scatter('mat1', mat1)
    vw.execute('mat3=mat1.dot(mat2)')
    return vw.gather('mat3', block=True)

a = np.matrix('1 2 3; 4 5 6; 7 8 9')
b = np.matrix('4 5 6; 7 8 9; 10 11 12')
paralleldot(drctview, a,b)

matrix([[ 48,  54,  60],
        [111, 126, 141],
        [174, 198, 222]])

#### 分散式計算兩個矩陣的內積

In [57]:
import numpy as np
from IPython.parallel import Client

ndim = 5
mat1 = np.random.randn(ndim, ndim)
mat2 = np.random.randn(ndim, ndim)
mat3 = np.dot(mat1,mat2)
mat3

array([[ 4.20395343,  1.77607694,  2.32612554,  1.57788378,  1.14594994],
       [-3.14555705, -2.72992455, -0.33495061, -3.11369927, -3.50043561],
       [-3.35040564,  1.84704767, -0.02498569, -0.144489  , -2.20582268],
       [-2.99338409, -0.76743994,  0.26960697, -2.06424304, -3.18198449],
       [ 3.56282324, -1.80239355, -0.90958053,  1.72901511,  1.4265735 ]])

In [58]:
clnt = Client()
clnt.ids

[0, 1, 2, 3]

In [59]:
dvw = clnt[:]
dvw.execute('import numpy as np', block=True)

<AsyncResult: execute:finished>

In [60]:
dvw.push(dict(a=mat1, b=mat2), block=True)
rslt = dvw.execute('mat3 = np.dot(a,b); print(mat3)', block=True)
rslt.display_outputs()

[stdout:0] 
[[ 4.20395343  1.77607694  2.32612554  1.57788378  1.14594994]
 [-3.14555705 -2.72992455 -0.33495061 -3.11369927 -3.50043561]
 [-3.35040564  1.84704767 -0.02498569 -0.144489   -2.20582268]
 [-2.99338409 -0.76743994  0.26960697 -2.06424304 -3.18198449]
 [ 3.56282324 -1.80239355 -0.90958053  1.72901511  1.4265735 ]]
[stdout:1] 
[[ 4.20395343  1.77607694  2.32612554  1.57788378  1.14594994]
 [-3.14555705 -2.72992455 -0.33495061 -3.11369927 -3.50043561]
 [-3.35040564  1.84704767 -0.02498569 -0.144489   -2.20582268]
 [-2.99338409 -0.76743994  0.26960697 -2.06424304 -3.18198449]
 [ 3.56282324 -1.80239355 -0.90958053  1.72901511  1.4265735 ]]
[stdout:2] 
[[ 4.20395343  1.77607694  2.32612554  1.57788378  1.14594994]
 [-3.14555705 -2.72992455 -0.33495061 -3.11369927 -3.50043561]
 [-3.35040564  1.84704767 -0.02498569 -0.144489   -2.20582268]
 [-2.99338409 -0.76743994  0.26960697 -2.06424304 -3.18198449]
 [ 3.56282324 -1.80239355 -0.90958053  1.72901511  1.4265735 ]]
[stdout:3] 
[[ 4

In [61]:
dot_product = dvw.pull('mat3', block=True)
print(dot_product)

[array([[ 4.20395343,  1.77607694,  2.32612554,  1.57788378,  1.14594994],
       [-3.14555705, -2.72992455, -0.33495061, -3.11369927, -3.50043561],
       [-3.35040564,  1.84704767, -0.02498569, -0.144489  , -2.20582268],
       [-2.99338409, -0.76743994,  0.26960697, -2.06424304, -3.18198449],
       [ 3.56282324, -1.80239355, -0.90958053,  1.72901511,  1.4265735 ]]), array([[ 4.20395343,  1.77607694,  2.32612554,  1.57788378,  1.14594994],
       [-3.14555705, -2.72992455, -0.33495061, -3.11369927, -3.50043561],
       [-3.35040564,  1.84704767, -0.02498569, -0.144489  , -2.20582268],
       [-2.99338409, -0.76743994,  0.26960697, -2.06424304, -3.18198449],
       [ 3.56282324, -1.80239355, -0.90958053,  1.72901511,  1.4265735 ]]), array([[ 4.20395343,  1.77607694,  2.32612554,  1.57788378,  1.14594994],
       [-3.14555705, -2.72992455, -0.33495061, -3.11369927, -3.50043561],
       [-3.35040564,  1.84704767, -0.02498569, -0.144489  , -2.20582268],
       [-2.99338409, -0.76743994,

In [62]:
np.allclose(mat3, dot_product[0])

True

In [63]:
np.allclose(dot_product[0], dot_product[1])

True

In [64]:
np.allclose(dot_product[1], dot_product[2])

True

### 8.4.4 支援使用 資料庫 做 requests and results 的緩存

In [65]:
from IPython import parallel
from datetime import datetime, timedelta
clients = parallel.Client()
incomplete_task = clients.db_query({'complete' : None}, keys=['msg_id', 'started'])
incomplete_task[:3]

[{'msg_id': 'd49e087f-39d36ee146959ed92689bc77',
  'started': datetime.datetime(2017, 3, 12, 23, 2, 25, 445677, tzinfo=tzoffset(None, 28800))},
 {'msg_id': '33e36620-a5d0b950a7a081dfc93f118c',
  'started': datetime.datetime(2017, 3, 12, 23, 2, 26, 118085, tzinfo=tzoffset(None, 28800))},
 {'msg_id': '1fdcbdfd-85a8f2b55697f77d050ff2c4',
  'started': datetime.datetime(2017, 3, 12, 23, 2, 26, 129869, tzinfo=tzoffset(None, 28800))}]

In [66]:
one_hourago = datetime.now() - timedelta(1./24)
tasks_started_hourago = clients.db_query({'started' : {'$gte' : one_hourago },'client_uuid' : clients.session.session})
tasks_started_hourago

[]

In [67]:
tasks_started_hourago_other_client = clients.db_query({'started' : {'$gte' : one_hourago }, 'client_uuid' : {'$ne' : clients.session.session}})
tasks_started_hourago_other_client

[{'client_uuid': 'ded0fe08-8e38be4e829212069a35d647',
  'completed': datetime.datetime(2017, 3, 12, 15, 2, 25, 558529, tzinfo=tzutc()),
  'content': {},
  'engine_uuid': 'fa334c62-31f544dba1526d5363f6c3ef',
  'error': None,
  'execute_input': None,
  'execute_result': None,
  'header': {'date': datetime.datetime(2017, 3, 12, 15, 2, 25, 442481, tzinfo=tzutc()),
   'msg_id': 'd49e087f-39d36ee146959ed92689bc77',
   'msg_type': 'apply_request',
   'session': 'ded0fe08-8e38be4e829212069a35d647',
   'username': 'wei',
   'version': '5.0'},
  'metadata': {},
  'msg_id': 'd49e087f-39d36ee146959ed92689bc77',
  'queue': 'mux',
  'received': datetime.datetime(2017, 3, 12, 23, 2, 25, 563395, tzinfo=tzoffset(None, 28800)),
  'resubmitted': None,
  'result_content': {'status': 'ok'},
  'result_header': {'date': datetime.datetime(2017, 3, 12, 15, 2, 25, 558529, tzinfo=tzutc()),
   'msg_id': '2835454a-a68dae60a0d495273c8c0bc0',
   'msg_type': 'apply_reply',
   'session': 'fa334c62-31f544dba1526d5363f6

In [68]:
uuids_of_3_n_4 = map(clients._engines.get, (3,4))
uuids_of_3_n_4

In [69]:
# headers_of_3_n_4 = clients.db_query({'engine_uuid' : {'$in' : uuids_of_3_n_4 }}, keys='result_header')
# headers_of_3_n_4

### 8.4.5 在 IPython 中使用 MPI
```
from IPython.parallel import Client
c = Client(profile='mpi')
view = c[:]
view.activate() # enable magics
view.run('psum.py')
view.scatter('a',np.arange(16,dtype='float'))
view['a']
%px totalsum = psum(a)
view['totalsum']
```

### 8.4.6 管理任務之間的 依賴關係
- 函數 依賴關係
 - @require
   - @require('pandas', 'numpy')
 - @depend
   - @depend(find_operating_system, 'linux')
- 圖 依賴關係
 - Dependency 類別 與 屬性
   - any | all
   - success | failure
   - after
   - follow
   - timeout
- 不可能滿足的 依賴關係
 - 評估是否該觸發 ImpossibleDependency 例外

In [70]:
import IPython.parallel as ipp
clients = ipp.Client()
lbview = clients.load_balanced_view()

task_fail = lbview.apply_async(lambda : 1/0)
task_success = lbview.apply_async(lambda : 'success')
clients.wait()

print("Fail task executed on %i" % task_fail.engine_id)
print("Success task executed on %i" % task_success.engine_id)

with lbview.temp_flags(after=task_success):
    print(lbview.apply_sync(lambda : 'Perfect'))

with lbview.temp_flags(follow=ipp.Dependency([task_fail, task_success], failure=True)):
    lbview.apply_sync(lambda : "impossible")

with lbview.temp_flags(after=Dependency([task_fail, task_success], failure=True, success=False)):
    lbview.apply_sync(lambda : "impossible")

def execute_print_engine(**flags):
    for idx in range(4):
        with lbview.temp_flags(**flags):
            task = lbview.apply_async(lambda : 'Perfect')
            task.get()
            print("Task Executed on %i" % task.engine_id)

execute_print_engine(follow=Dependency([task_fail, task_success], all=False))
execute_print_engine(after=Dependency([task_fail, task_success], all=False))
execute_print_engine(follow=Dependency([task_fail, task_success], all=False, failure=True, success=False))
execute_print_engine(follow=Dependency([task_fail, task_success], all=False, failure=True))

Fail task executed on 0
Success task executed on 1
Perfect


RemoteError: ImpossibleDependency()

- DAG 依賴關係與NetworkX函數庫
  - 有向無環圖 (Directed Acyclic Graph, DAG)

### 8.4.7 用 Amazon EC2 的 StarCluster 啟動 IPython

## 8.5 IPython 數據安全措施

### 8.5.1 常用的 並行編程方法
- 並行
  - 經典(IPC)問題
    - Dead lock
      - 四個條件
        - Mutual exclusion
        - Hold and wait
        - Non-preemption
        - Circular wait
    - Starvation
    - Race condition
  - 並行編程
    - 可能式分佈在多個CPU 或者多台機器上
- 並發編程
  - Workers 的數量多於 CPU 的數量，所以必須要透過 scheduler 來分配資源
- 分佈式編程
  - workers 在不同的機器上執行
- Python 多進程
- Python 多線程

#### Process 類別

In [71]:
import multiprocessing as mpcs
import random
import string

output_queue = mpcs.Queue()

def strings_random(length, output_queue):
    generated_string = ''.join(random.choice(string.ascii_lowercase + string.ascii_uppercase + string.digits)
        for i in range(length))
    output_queue.put(generated_string)

procs = [mpcs.Process(target=strings_random, args=(8, output_queue)) for i in range(7)]

for proc in procs:
    proc.start()

for proc in procs:
    proc.join()

results = [output_queue.get() for pro in procs]
print(results)

['Lu5jNo9w', 'iPn8Vh3y', '6wUrKxdZ', '5hsUa5eP', 'GH2eWorX', '7mb8zlhI', 'zFVgHbNq']


#### 加上 位置參數 以便排序

In [72]:
import multiprocessing as mpcs
import random
import string

output_queue = mpcs.Queue()

def strings_random(length, position, output_queue):
    generated_string = ''.join(random.choice(string.ascii_lowercase    + string.ascii_uppercase + string.digits)
        for i in range(length))
    output_queue.put((position, generated_string))

procs = [mpcs.Process(target=strings_random, args=(5, pos, output_queue)) for pos in range(4)]

for proc in procs:
    proc.start()

for proc in procs:
    proc.join()

results = [output_queue.get() for pro in procs]
results.sort()
results = [rslt[1] for rslt in results]
print(results)

['xdjDX', 'wP8JG', 'aDwCB', 's36BR']


#### Pooling

In [73]:
import multiprocessing as mpcs
import random
import string

def calc_square(n):
    return n**2

pool_processes = mpcs.Pool(processes=5)

results = [pool_processes.apply(calc_square, args=(i,)) for i in range(1,10)]
print(results)

pool_processes = mpcs.Pool(processes=5)
results = pool_processes.map(calc_square, range(1,10))
print(results)

[1, 4, 9, 16, 25, 36, 49, 64, 81]
[1, 4, 9, 16, 25, 36, 49, 64, 81]


#### Thread 

In [74]:
import threading
import time

class demoThread (threading.Thread):
    def __init__(self, threadID, name, ctr):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.ctr = ctr
    def run(self):
        print("Start of The Thread: " + self.name)
        print_time(self.name, self.ctr, 8)
        print("Thread about to Exit:" + self.name)

def print_time(threadName, delay, counter):
        while counter:
            time.sleep(delay)
            print("%s: %s" % (threadName, time.ctime(time.time())))
            counter -= 1

thrd1 = demoThread(1, "FirstThread", 4)
thrd2 = demoThread(2, "SecondThread", 5)

thrd1.start()
thrd2.start()

print("Main Thread exits")

Start of The Thread: FirstThread
Main Thread exits
Start of The Thread: SecondThread
FirstThread: Sun Mar 12 23:03:31 2017
SecondThread: Sun Mar 12 23:03:32 2017
FirstThread: Sun Mar 12 23:03:35 2017


#### Thread lock

In [ ]:
import threading
import time
class demoThread (threading.Thread):
    def __init__(self, threadID, name, ctr):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.ctr = ctr
    def run(self):
        print("Start of The Thread: " + self.name)
        threadLock.acquire()
        print_time(self.name, self.ctr, 8)
        print("Thread about to Exit:" + self.name)
        threadLock.release()

def print_time(threadName, delay, counter):
        while counter:
            time.sleep(delay)
            print("%s: %s" % (threadName, time.ctime(time.time())))
            counter -= 1

threadLock = threading.Lock()
thrds = []

thrd1 = demoThread(1, "FirstThread", 4)
thrd2 = demoThread(2, "SecondThread", 5)

thrd1.start()
thrd2.start()

thrds.append(thrd1)
thrds.append(thrd2)

for thrd in thrds:
    thrd.join()

print("Main Thread Exits")

### 8.5.2 在 Python 中演示基於 Hadoop 的 MapReduce

#### map

In [ ]:
#!/usr/bin/env python

import sys

for line in sys.stdin:
    try:
        line = line.strip()
        # split the line into words
        words = line.split()
        # increase counters    
        if words[0] == "WARC-Target-URI:" :
            uri = words[1].split("/")
            print '%s\t%s' % (uri[0]+"//"+uri[2], 1)
    except Exception:
        ""
#hadoop jar /usr/local/apache/hadoop2/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar -file /mapper.py    -mapper /mapper.py -file /reducer.py   -reducer /reducer.py -input /text.txt -output /output


#### reduce

In [ ]:
#!/usr/bin/env python

from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

for line in sys.stdin:
    line = line.strip()

    word, count = line.split('\t', 1)

    try:
        count = int(count)
    except ValueError:
        continue

    if current_word == word:
        current_count += count
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

if current_word == word:
    print '%s\t%s' % (current_word, current_count)

### 8.5.3 在 Python 中運行 Spark
PySpark